In [3]:
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# download ml-100k.csv
!wget -q http://media.sundog-soft.com/es/ml-100k.zip
#wget: wget is a command-line utility for downloading files from the internet. It's commonly used in Unix-like operating systems (e.g., Linux) to fetch files from web servers.

In [7]:
# save the csv file in the working directory
import os
import requests
import zipfile
import io

# URL of the ZIP file to download
zip_url = "http://media.sundog-soft.com/es/ml-100k.zip"

# Send an HTTP GET request to download the ZIP file
response = requests.get(zip_url)

# Check if the request was successful
if response.status_code == 200:
    # Get the file name from the URL
    file_name = os.path.basename(zip_url)

    # Determine the current working directory
    current_directory = os.getcwd()

    # Specify the path to save the ZIP file in the current directory
    zip_save_path = os.path.join(current_directory, file_name)

    # Save the downloaded ZIP file to the specified path
    with open(zip_save_path, "wb") as file:# wb means write binary
        file.write(response.content)

    print(f"ZIP file saved to: {zip_save_path}")

    # Extract the contents of the ZIP file
    with zipfile.ZipFile(zip_save_path, 'r') as zip_ref:
        zip_ref.extractall(current_directory)

    print(f"ZIP file extracted to: {current_directory}")

    # Now you can process the extracted CSV file (assuming it's named "data.csv")
    extracted_csv_file = os.path.join(current_directory, "ml-100k/u.data")

    if os.path.isfile(extracted_csv_file):
        # You can work with the extracted CSV file here
        print(f"Extracted CSV file found at: {extracted_csv_file}")
    else:
        print(" file not found in the extracted contents.")
else:
    print("Failed to download the ZIP file.")

ZIP file saved to: /content/ml-100k.zip
ZIP file extracted to: /content
Extracted CSV file found at: /content/ml-100k/u.data


In [8]:
from pyspark import SparkConf, SparkContext
import collections

conf = SparkConf().setMaster("local").setAppName("RatingsHistogram")
sc = SparkContext(conf = conf)

lines = sc.textFile("/content/ml-100k/u.data")
ratings = lines.map(lambda x: x.split()[2])
result = ratings.countByValue()

sortedResults = collections.OrderedDict(sorted(result.items()))
for key, value in sortedResults.items():
    print("%s %i" % (key, value))


1 6110
2 11370
3 27145
4 34174
5 21201
